## Setup and Import Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import nest_asyncio
from llama_index.core.schema import Document
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval import TruLlama, Tru
from trulens.core import Feedback, FeedbackMode
from trulens_eval import OpenAI as fOpenAI
from utils import build_sentence_window_index, get_sentence_window_query_engine
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [5]:
tru = Tru()
tru.reset_database()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [6]:
documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [7]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [11]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [12]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [13]:
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model_name="BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

Loading llama_index.core.storage.kvstore.simple_kvstore from sentence_index\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from sentence_index\index_store.json.


In [14]:
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [15]:
output = sentence_window_engine.query("How do you create your AI portfolio?")

print(output.response)

Start by working on projects that show progress over time from simple to complex undertakings. Communicate the value of your work clearly to others to gain their trust and support. As you complete projects, be able to explain clearly what you accomplished to convince others of your capabilities. Additionally, consider taking on leadership roles in projects to demonstrate your ability to lead and apply deep technical insights. By building a portfolio that showcases skill progression and a variety of projects, you will be better positioned when looking for a job in AI.


## Feedback Functions

In [16]:
nest_asyncio.apply()

In [17]:
provider = fOpenAI()

### 1. Answer Relevance

In [18]:
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [19]:
context_selection = TruLlama.select_source_nodes().node.text

In [20]:
f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [21]:
f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


### 3. Groundedness

In [25]:
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons,
        name="Groundedness"
    )
    .on(context_selection)
    .on_output()
)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of RAG Application

In [26]:
tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

In [28]:
eval_questions = []

with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [29]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [30]:
eval_questions.append("How can I be successful in AI?")

In [31]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [32]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [33]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [34]:
records.head()

,app_name,app_version,app_id,app_json,type,record_id,input,output,tags,record_json,...,Answer Relevance,Answer Relevance_calls,Answer Relevance feedback cost in USD,Groundedness,Groundedness_calls,Groundedness feedback cost in USD,latency,total_tokens,total_cost,cost_currency
0,App_1,base,app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_ee38f7f27a552547677be06ceda65335,How can I be successful in AI?,"To be successful in AI, it is important to foc...",-,{'record_id': 'record_hash_ee38f7f27a552547677...,...,NaN,NaN,NaN,NaN,NaN,NaN,3.053206,1831,0.002810,USD
1,App_1,base,app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_0ee54d7edac650165f27e12f13b00f57,Is it normal to find parts of AI challenging?,It is normal to find parts of AI challenging.,-,{'record_id': 'record_hash_0ee54d7edac650165f2...,...,1.000000,[{'args': {'prompt': 'Is it normal to find par...,0.0,NaN,NaN,NaN,2.047881,2118,0.003182,USD
2,App_1,base,app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_96bf1e249c1383d1433ebd0437daa9da,What are some common challenges in AI?,Some common challenges in AI include experienc...,-,{'record_id': 'record_hash_96bf1e249c1383d1433...,...,1.000000,[{'args': {'prompt': 'What are some common cha...,0.0,NaN,NaN,NaN,2.370252,2173,0.003285,USD
3,App_1,base,app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_e15da715cb9e98c176c83b6ec9c6af08,What is the first step to becoming good at AI?,Learning foundational technical skills.,-,{'record_id': 'record_hash_e15da715cb9e98c176c...,...,0.666667,[{'args': {'prompt': 'What is the first step t...,0.0,0.5,[{'args': {'source': 'Here’s a framework for c...,0.000032,1.916715,1696,0.002547,USD
4,App_1,base,app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e,"{'tru_class_info': {'name': 'TruLlama', 'modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_7d48fcd34874a8ae59416612b5ae9583,Who are some accomplished individuals who have...,"Former Facebook COO Sheryl Sandberg, U.S. firs...",-,{'record_id': 'record_hash_7d48fcd34874a8ae594...,...,1.000000,[{'args': {'prompt': 'Who are some accomplishe...,0.0,0.5,[{'args': {'source': 'PAGE 37 Overcoming Impos...,0.000087,2.490092,2185,0.003299,USD


In [35]:
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Answer Relevance
0,How can I be successful in AI?,"To be successful in AI, it is important to focus on building a strong professional network or community, rather than just networking. Additionally, developing good habits in various aspects of life such as learning, personal relationships, and self-care can contribute to long-term success. Furthermore, aiming to lift others during your own journey and practicing altruism can lead to better outcomes. Building a portfolio of projects that show skill progression, from simple to complex undertakings, is crucial when looking for a job in AI. Effective communication, leadership skills, and the ability to work in teams are also key factors in achieving success in the field of AI.",NaN,NaN
1,Is it normal to find parts of AI challenging?,It is normal to find parts of AI challenging.,NaN,1.000000
2,What are some common challenges in AI?,"Some common challenges in AI include experiencing imposter syndrome, struggling with technical complexities, facing difficulties in understanding and implementing algorithms, finding research papers challenging to read, making mistakes in tuning neural network hyperparameters, and feeling overwhelmed by the complexity of the field.",NaN,1.000000
3,What is the first step to becoming good at AI?,Learning foundational technical skills.,0.500000,0.666667
4,Who are some accomplished individuals who have experienced imposter syndrome?,"Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.",0.500000,1.000000
5,What is imposter syndrome and how does it relate to AI?,"Imposter syndrome is when someone doubts their abilities and feels like a fraud, despite their success or accomplishments in a particular field. In the context of AI, newcomers to the field may experience imposter syndrome, questioning if they truly belong in the AI community even if they have achieved success. It is highlighted that imposter syndrome is common among many individuals, including accomplished figures, and that it should not discourage anyone from growing in AI. The message conveyed is that experiencing challenges and doubts in AI is normal, and it is important to support and welcome everyone who wants to be part of the AI community.",0.777778,1.000000
6,How can altruism be beneficial in building a career?,Helping others during each step of your own career journey can lead to better outcomes for yourself.,0.500000,0.666667
7,What are some good habits to develop for a successful career?,"Good habits to develop for a successful career include working in teams effectively, building strong interpersonal and communication skills, focusing on networking and building a community rather than just personal connections, maintaining personal discipline in areas like eating, exercise, sleep, and self-care, and practicing altruism by helping others along the way.",0.500000,1.000000
8,What is the importance of networking in AI?,"Networking is crucial in AI as it helps individuals build a strong professional network that can provide support, guidance, and opportunities. By connecting with others in the field, individuals can gain valuable insights, collaborate on projects, and stay updated on industry trends. Building a community and fostering genuine relationships can propel one's career forward and create a supportive environment for growth and success in the field of AI.",0.416667,1.000000
9,How can teamwork contribute to success in AI?,"Teamwork can contribute to success in AI by enabling individuals to collaborate effectively, influence others, and be influenced by team members. The ability to work in teams allows for sharing of ideas, pooling of expertise, and leveraging diverse perspectives to tackle complex AI projects. Interpersonal and communication skills play a crucial role in fostering a collaborative environment wh

In [36]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,
App_1,base,0.878788,0.603535,3.380495,0.002899


In [38]:
# tru.run_dashboard()